In [1]:
%load_ext autoreload
%autoreload 2

# How to return artifacts from a tool
_Reference_: [https://python.langchain.com/docs/how_to/tool_artifacts/](https://python.langchain.com/docs/how_to/tool_artifacts/)

# Defining the tool

In [2]:
import random
from typing import List, Tuple

from langchain_core.tools import tool

In [3]:
@tool(response_format="content_and_artifact")
def generate_random_ints(min: int, max: int, size: int) -> Tuple[str, List[int]]:
    """Generate size random ints in the range [min, max]."""
    array = [random.randint(min, max) for _ in range(size)]
    content = f"Successfully generated array of {size} random ints in [{min}, {max}]."
    return content, array

# Invoking the tool with ToolCall

In [4]:
generate_random_ints.invoke({"min": 0, "max": 9, "size": 10})

'Successfully generated array of 10 random ints in [0, 9].'

In [5]:
generate_random_ints.invoke(
    {
        "name": "generate_random_ints",
        "args": {"min": 0, "max": 9, "size": 10},
        "id": "123",  # required
        "type": "tool_call",  # required
    }
)

ToolMessage(content='Successfully generated array of 10 random ints in [0, 9].', name='generate_random_ints', tool_call_id='123', artifact=[7, 3, 3, 8, 7, 6, 4, 8, 5, 9])

# Using with a model

In [6]:
def load_env_to_dict(file_path):
    env_dict = {}
    with open(file_path, "r") as file:
        for line in file:
            # Remove whitespace and ignore comments or empty lines
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            # Split the line into key and value
            key, value = line.split("=", 1)
            env_dict[key.strip()] = value.strip()
    return env_dict

In [7]:
file_path = "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
env_variables = load_env_to_dict(file_path)

In [8]:
import os

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = env_variables["HUGGINGFACE_API_KEY"]

In [9]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [10]:
llm = ChatHuggingFace(
    llm=HuggingFaceEndpoint(
        repo_id="HuggingFaceH4/zephyr-7b-beta",
        task="text-generation",
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
        temperature=0.1,
    ),
    verbose=True,
)

In [11]:
llm_with_tools = llm.bind_tools([generate_random_ints])

In [12]:
ai_msg = llm_with_tools.invoke("Lấy cho tôi 6 số nguyên ngẫu nhiên nhỏ hơn 25")
ai_msg.tool_calls

[{'name': 'generate_random_ints',
  'args': {'size': 6, 'min': 1, 'max': 25},
  'id': '0',
  'type': 'tool_call'}]

In [13]:
generate_random_ints.invoke(ai_msg.tool_calls[0])

ToolMessage(content='Successfully generated array of 6 random ints in [1, 25].', name='generate_random_ints', tool_call_id='0', artifact=[6, 5, 20, 19, 3, 1])

If we just pass in the tool call args, we'll only get back the content:

In [14]:
generate_random_ints.invoke(ai_msg.tool_calls[0]["args"])

'Successfully generated array of 6 random ints in [1, 25].'

If we wanted to declaratively create a chain, we could do this:

In [15]:
from operator import attrgetter

In [22]:
chain = llm_with_tools | attrgetter("tool_calls") | generate_random_ints.map()

chain.invoke("Cho tôi một số ngẫu nhiên trong khoảng từ 1 đến 5")

[ToolMessage(content='Successfully generated array of 3 random ints in [1, 5].', name='generate_random_ints', tool_call_id='0', artifact=[4, 3, 1])]

# Creating from BaseTool class
- If you want to create a `BaseTool` object directly, instead of decorating a function with `@tool`, you can do so like this:

In [23]:
from langchain_core.tools import BaseTool

In [24]:
class GenerateRandomFloats(BaseTool):
    name: str = "generate_random_floats"
    description: str = "Generate size random floats in the range [min, max]."
    response_format: str = "content_and_artifact"

    ndigits: int = 2

    def _run(self, min: float, max: float, size: int) -> Tuple[str, List[float]]:
        range_ = max - min
        array = [
            round(min + (range_ * random.random()), ndigits=self.ndigits)
            for _ in range(size)
        ]
        content = f"Generated {size} floats in [{min}, {max}], rounded to {self.ndigits} decimals."
        return content, array

    # Optionally define an equivalent async method

    # async def _arun(self, min: float, max: float, size: int) -> Tuple[str, List[float]]:
    #     ...

In [25]:
rand_gen = GenerateRandomFloats(ndigits=4)
rand_gen.invoke({"min": 0.1, "max": 3.3333, "size": 3})

'Generated 3 floats in [0.1, 3.3333], rounded to 4 decimals.'

In [26]:
rand_gen.invoke(
    {
        "name": "generate_random_floats",
        "args": {"min": 0.1, "max": 3.3333, "size": 3},
        "id": "123",
        "type": "tool_call",
    }
)

ToolMessage(content='Generated 3 floats in [0.1, 3.3333], rounded to 4 decimals.', name='generate_random_floats', tool_call_id='123', artifact=[1.0532, 0.7891, 0.3285])